In [11]:
from Bio import Entrez
from datetime import datetime
import json
import xmltodict
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import seaborn as sns
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sqlalchemy import create_engine, Column, Integer, String, DateTime, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session


# Data Extraction from PubMed

In [2]:
def data_extractor(json_data):
    data = json_data["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"]

    date = pd.to_datetime(data["DateRevised"]["Year"] +
                        data["DateRevised"]["Month"] + data["DateRevised"]["Day"])
    
    
    if isinstance(data["Article"]["ArticleTitle"], dict):
        title = data["Article"]["ArticleTitle"]["#text"]
    else:
        title = data["Article"]["ArticleTitle"]

    authors = []
    author_data = data["Article"]["AuthorList"]["Author"]
    if type(author_data) == list:
        for data_dict in author_data:
            name = data_dict["ForeName"] + " " + data_dict["LastName"]
            authors.append(name)
    elif type(author_data) == dict:
        name = author_data["ForeName"] + " " + author_data["LastName"]
        authors.append(name)

    keywords = []
    keywords_path = data["KeywordList"]["Keyword"]
    if isinstance(keywords_path, list):
        for keyword in data["KeywordList"]["Keyword"]:
            words = keyword["#text"]
            keywords.append(words)
    elif isinstance(keywords_path, dict):
            keywords = keywords_path["#text"]
        

    abstract = data["Article"]["Abstract"]["AbstractText"]
    final_abstract = False
    if isinstance(abstract, list):
        for text in abstract:
            if final_abstract == False:
                final_abstract = text["#text"]
            else:
                final_abstract += f"\n {text['#text']}"
    elif isinstance(abstract, dict):
        final_abstract = abstract["#text"]
    else:
        final_abstract = abstract

    locator_format = False
    locator_number = False
    locators = data["Article"]["ELocationID"]
    if isinstance(locators, list):
        for locator in locators:
            if locator["@EIdType"] == "doi": #encontrar forma de que si no existe doi poner el primero
                locator_format = locator["@EIdType"]
                locator_number = locator["#text"]
    else:
        locator_format = locators["@EIdType"]
        locator_number = locators["#text"]

    df = pd.DataFrame([[data["PMID"]["#text"], title, date, data["Article"]["Journal"]["Title"], data["Article"]["Journal"]["ISOAbbreviation"],
                    authors, final_abstract, keywords, locator_format, locator_number]],
                    columns=["PMID", "Title", "Date", "Journal", "Journal_abreviation", "All_authors", "Abstract", "Keywords", "Locator_format", "Locator_number"])
    df = df.set_index("PMID")
    df.index = df.index.astype(int)
    return df

In [8]:
# Configura tu dirección de correo electrónico (es necesario para usar la API)
Entrez.email = "josemanuelgonzalezfornell@gmail.com"

# Obtén la fecha actual y calcula la fecha de hace un año
fecha_actual = datetime.now()
fecha_hace_un_anio = fecha_actual.replace(year=fecha_actual.year - 1)

# Convierte las fechas al formato necesario para la búsqueda en PubMed
fecha_actual_str = fecha_actual.strftime("%Y/%m/%d")
fecha_hace_un_anio_str = fecha_hace_un_anio.strftime("%Y/%m/%d")

max_results=2
retmax=2
all_results = []

# Realiza la búsqueda en PubMed
for retstart in range(0, max_results, retmax):
    handle = Entrez.esearch(db="pubmed", term=f'"{fecha_hace_un_anio_str}"[Date - Publication] : "{fecha_actual_str}"[Date - Publication]', retmax=retmax, retstart=retstart)
    record = Entrez.read(handle)
    handle.close()
    all_results.extend(record["IdList"])


handle = Entrez.efetch(db="pubmed", id=all_results, retmode="xml", rettype="abstract")
record = handle.read()
handle.close()

df_final = pd.DataFrame()

counter = 1

# Descarga la información de cada paper
for id_paper in all_results:
    handle = Entrez.efetch(db="pubmed", id=id_paper)
    record = handle.read()
    handle.close()

    # Convierte XML a JSON usando xmltodict
    json_data = xmltodict.parse(record)
    print(f"start paper {counter}/{max_results}")
    try:
        df = data_extractor(json_data)
        df_final = pd.concat([df_final, df])
        print(f"finish paper {counter}/{max_results}")
        counter += 1
    except KeyError:
        print(f"paper {counter}/{max_results} ignored")
        counter += 1
        continue        





start paper 1/2
finish paper 1/2
start paper 2/2
finish paper 2/2


In [4]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, 38153010 to 38153002
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Title                7 non-null      object        
 1   Date                 7 non-null      datetime64[ns]
 2   Journal              7 non-null      object        
 3   Journal_abreviation  7 non-null      object        
 4   All_authors          7 non-null      object        
 5   Abstract             7 non-null      object        
 6   Keywords             7 non-null      object        
 7   Locator_format       7 non-null      object        
 8   Locator_number       7 non-null      object        
dtypes: datetime64[ns](1), object(8)
memory usage: 560.0+ bytes


# BBDD create and load

In [5]:
df_to_ddbb = df_final.copy()
df_to_ddbb["All_authors"] = df_to_ddbb["All_authors"].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')
df_to_ddbb["Keywords"] = df_to_ddbb["Keywords"].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')

engine = create_engine(f'sqlite:///../data/processed/Pubmed_DDBB.db')

Base = declarative_base()

# Define la tabla Main (puedes omitir esto si ya existe)
class Main(Base):
    __tablename__ = 'Main'
    PMID = Column(Integer, primary_key=True)
    Title = Column(String)
    Date = Column(DateTime)
    Journal = Column(String)
    Journal_abreviation = Column(String)
    All_authors = Column(String)
    Abstract = Column(String)
    Keywords = Column(String)
    Locator_format = Column(String)
    Locator_number = Column(String)

# Crea la tabla en la base de datos
Base.metadata.create_all(engine)


df_to_ddbb.to_sql('Main', con=engine, index=True, if_exists='replace')

/var/folders/r1/hd_91nzn1yb8kh1vq1c7gt2w0000gn/T/ipykernel_8811/1434489641.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


7

# Clean data  
## Stopwords removal

In [24]:
engine = create_engine(f'sqlite:///../data/processed/Pubmed_DDBB.db')
session = Session(engine)

query = text("SELECT PMID, Abstract, Journal FROM Main")

result = session.execute(query).fetchall()
df_abstract = pd.DataFrame(result).set_index("PMID")

In [ ]:
#TODO Eliminar links
#TODO corregir texto antes de quitar stopsword
#TODO realizar un baseline con los diferentes modelos de textBlob

In [ ]:
signos = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\¿)|(\@)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")

def signs_clean(text):
    return signos.sub('', text.lower())

df_abstract.loc[:, "Abstract"] = df_abstract["Abstract"].apply(lambda x: signs_clean(x))
df_abstract.head()

,Journal,Abstract
PMID,,
38145486,Journal of medical Internet research,chatgpt may act research assistant help organi...
38145485,Journal of medical Internet research,acquired brain injury abi children adolescents...
38145484,JMIR rehabilitation and assistive technologies,promoting well-being older adults aging societ...
38145483,JMIR formative research,online pharmacies used less e-commerce sites g...
38145482,JMIR formative research,increasing numbers residency applications crea...


In [ ]:
english_stopwords = stopwords.words('english')

def remove_stopwords(df):
    return " ".join([word for word in df.split() if word not in english_stopwords])

df_abstract.loc[:, "Abstract"] = df_abstract["Abstract"].apply(lambda x: remove_stopwords(x))
df_abstract.head()

,Journal,Abstract
PMID,,
38145486,Journal of medical Internet research,chatgpt may act research assistant help organi...
38145485,Journal of medical Internet research,acquired brain injury abi children adolescents...
38145484,JMIR rehabilitation and assistive technologies,promoting well-being older adults aging societ...
38145483,JMIR formative research,online pharmacies used less e-commerce sites g...
38145482,JMIR formative research,increasing numbers residency applications crea...


## Stemming

In [ ]:
def stemmer(x):
    stemmer = SnowballStemmer('english')
    return " ".join([stemmer.stem(word) for word in x.split()])

df_abstract.loc[:, "Abstract"] = df_abstract["Abstract"].apply(lambda x: stemmer(x))
df_abstract.head()

,Journal,Abstract
PMID,,
38145486,Journal of medical Internet research,chatgpt may act research assist help organ dir...
38145485,Journal of medical Internet research,acquir brain injuri abi children adolesc lead ...
38145484,JMIR rehabilitation and assistive technologies,promot well-b older adult age societi requir n...
38145483,JMIR formative research,onlin pharmaci use less e-commerc site germani...
38145482,JMIR formative research,increa number resid applic creat challeng appl...


In [ ]:
from textblob import TextBlob

df_abstract.loc[:, "Abtract"] = df_abstract["Abstract"].apply(lambda x: TextBlob(x))
df_abstract.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33 entries, 38145486 to 38145447
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Journal   33 non-null     object
 1   Abstract  33 non-null     object
 2   Abtract   33 non-null     object
dtypes: object(3)
memory usage: 2.1+ KB


In [ ]:
dd = df_abstract["Abtract"].iloc[2]

In [ ]:
dd.correct()

TextBlob("promote well-b older adult age society require new slut one resource might use social root group active promote physics count stimuli engage root-assist group active may help slowdown physics count decline older adult current knowledge limit whether older adult engage group active humanlik social root whether expert post affect necessary precondit achieve intend effect group active pilot study aim first aim develop pilot observe code scheme root-assist group active self-report data engage mood nur home resin often difficult obtain exist observe instrument limit second aim investing older adult engage mood root-assist group active differ nur care home german-speak part switzerland develop observe system inspire exist tool structure observe engage mood older adult root-assist group active study older adult resin differ care home switzerland particle root-assist group active session observe use develop system data collect form video clip assess later regard engage direct gaze po